In [4]:
from pymongo import MongoClient
import pandas as pd
import time
import re
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import os
import requests
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("./dataset/dataset.csv")

# Filtering the number of companies, according to criterias we have


The filter process was split into 2 steps,  

1- First filter we used a mongo query to select form the database design companies that had raised more than 10M$, analysed in the first Jupyet Notebook.  

2- For the second filter we are going to be using foursquare API to narrow down our dataset, and get a final dataframe with only companies that meet our conditions:  
 -- we will be deleting any company(coordinates) that has no amenities around.  
 -- for that matter we will be creating additional column for each amenity with their number(how many are there)
 
    


In [8]:
load_dotenv()

True

In [9]:
token_fsq = os.getenv("token")

In [10]:
df.country_code.unique()


array(['USA', 'FRA', 'NLD', 'SGP', 'GBR', 'DEU', 'CHN', 'ESP', 'CAN',
       'FIN', 'IRL'], dtype=object)

In [11]:
#this is a test
usa_df = df[df["country_code"] == "USA"]
#test api with country_code FRA, bc it doesn't have too many values
fra_df = df[df["country_code"] == "FRA"]


# Starbucks
1- look for Starbucks cafes nearby, because everyone needs coffee.  
2- deleting location with no starbucks around.

     (setting the radius to 500 and limiting search to 50.)

In [12]:
def getNearbyStarbucks(lat, long, radius=500):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&chains=ab4c54c0-d68a-012e-5619-003048cad9da&exclude_all_chains=false&sort=DISTANCE&limit=50"        
    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    results = requests.get(url,headers=headers).json()["results"]
    
    return len(results)
     

In [13]:
for index, row in df.iterrows():
    df.at[index, "Starbucks"] = getNearbyStarbucks(row["latitude"],row["longitude"])
   

In [14]:
star_count = df["Starbucks"].value_counts()
#print(star_count.to_string())

     


In [15]:
df['Starbucks'].dtypes
df['Starbucks'] = df['Starbucks'].astype(int)

In [16]:
#pd.set_option('display.max_rows', 500)
#df


In [17]:
#deleting the locations with zero Starbucks nearby
df = df[df.Starbucks != 0]

# Nightlife

1- look for Bars nearby, because everyone in the company is between 25 and 40, and need some place to go party.  
2- deleting location with no bars around.  


 (setting the radius to 500 and limiting search to 50.)

In [18]:
def getNearbyNightlife(lat, long, radius=500):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories=13003&exclude_all_chains=false&sort=DISTANCE&limit=50"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    results = requests.get(url,headers=headers).json()["results"]
    
    return len(results)


In [19]:
for index, row in df.iterrows():
    df.at[index, "Nightlife"] = getNearbyNightlife(row["latitude"],row["longitude"])

In [20]:
df['Nightlife'] = df['Nightlife'].astype(int)

In [21]:
nl_count = df["Nightlife"].value_counts()
#print(nl_count.to_string())

In [22]:
df = df[df.Nightlife != 0]

In [23]:
city_count = df["country_code"].value_counts()
#print(city_count.to_string())

# Transportation
Third filter is Transportation, specifically train station,metro station and taxi.(sone of them offer shuttle to the airport)
--assuming that airports are generally located outside cities, by choosing an office near an airport,we are going to be discarding other criterias that can't be found near airports, such as schools,pet grooming services...
 


In [24]:
def getNearbyTransport(lat, long, radius=500):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories=19047%2C19049%2C19046&exclude_all_chains=false&sort=DISTANCE&limit=50"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    results = requests.get(url,headers=headers).json()["results"]
    
    return len(results)

In [25]:
for index, row in df.iterrows():
    df.at[index, "transportation"] = getNearbyTransport(row["latitude"],row["longitude"])

In [26]:
df['transportation'] = df['transportation'].astype(int)

In [27]:
df = df[df.transportation != 0]

In [28]:
transp_count = df["transportation"].value_counts()
#print(transp_count.to_string())
#df

# Preschool
30% of the company staff have at least 1 child.  
Then all the locations with 0 preschool around are going to be deleted.



In [29]:
def getNearbypreschool(lat, long, radius=1000):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories=12056&exclude_all_chains=false&sort=DISTANCE&limit=50"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    results = requests.get(url,headers=headers).json()["results"]
    
    return len(results)
         


In [30]:
for index, row in df.iterrows():
    df.at[index, "preschool"] = getNearbypreschool(row["latitude"],row["longitude"])

In [31]:
df['preschool'] = df['preschool'].astype(int)

In [32]:
ps_count = df["preschool"].value_counts()
#print(ps_count.to_string())

In [33]:
df = df[df.preschool != 0]

In [34]:
city_count = df["country_code"].value_counts()
#print(city_count.to_string())

# Vegan restaurants

In [35]:
def getNearbyVeganRest(lat, long, radius=500):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories=13377&exclude_all_chains=false&sort=DISTANCE&limit=50"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    results = requests.get(url,headers=headers).json()["results"]
    
    return len(results)
      

In [36]:
for index, row in df.iterrows():
    df.at[index, "veganRest"] = getNearbyVeganRest(row["latitude"],row["longitude"])

In [37]:
df['veganRest'] = df['veganRest'].astype(int)

In [38]:
vegan_count = df["veganRest"].value_counts()
#print(vegan_count.to_string())
#df

In [39]:
city_count = df["country_code"].value_counts()
print(city_count.to_string())

USA    141
GBR      5
SGP      2
ESP      2
FRA      1
CAN      1


In [40]:
df = df[df.veganRest != 0]

# Pet grooming service
The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.  
the radius is set to 1000, Dobby only needs to go there once a month.


In [41]:
def getNearbyPetService(lat, long, radius=1000):

    # create the API request URL
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&radius={radius}&categories=11134&exclude_all_chains=false&sort=DISTANCE"

    # make the GET request
    headers = {
    "accept": "application/json",
    "Authorization": token_fsq
    }
    
    results = requests.get(url,headers=headers).json()["results"]
    
    return len(results)
      

In [42]:
for index, row in df.iterrows():
    df.at[index, "petService"] = getNearbyPetService(row["latitude"],row["longitude"])

In [43]:
df['petService'] = df['petService'].astype(int)

In [44]:
Ps_count = df["petService"].value_counts()
#print(Ps_count.to_string())

In [45]:
df = df[df.petService != 0]

In [46]:
city_count = df["country_code"].value_counts()
#print(city_count.to_string())

In [47]:
city_count = df["city"].value_counts()
#print(city_count.to_string())

In [48]:
test1 = df[~df.country_code.str.contains("USA")]
test1


,name,city,latitude,longitude,country_code,Starbucks,Nightlife,transportation,preschool,veganRest,petService
55,Wonga,London,51.519204,-0.162610,GBR,3,42,14,1,2,1
64,mig33,Singapore,37.580304,-122.343679,SGP,1,11,5,7,1,2
149,Globant,London,51.520510,-0.095220,GBR,2,50,6,5,2,1
287,Nexway,Madrid,40.414299,-3.703307,ESP,4,50,5,4,12,2
288,Nexway,Singapore,1.307505,103.829500,SGP,6,50,1,4,3,1
314,Netbiscuits,London,51.513248,-0.154738,GBR,3,50,6,1,3,1


In [90]:
df.to_csv("./dataset/world_dataset.csv", index = False)


In [91]:
new_df = pd.read_csv("./dataset/world_dataset.csv")

# Dataset with all needed amenities

In [92]:
new_df.sample()

,name,city,latitude,longitude,country_code,Starbucks,Nightlife,transportation,preschool,veganRest,petService
62,Bestofmedia Group,Culver City,34.02213,-118.395743,USA,1,12,1,7,8,7


Now that we have a dataset with all the criterias we need, we are going to check the total amenities in each location, to have a look at the density : we want our office to be in a dense location.  
1- we are getting the total amenities for each location  
2- we are checking the ration...


In [93]:
new_df["Total"] = (new_df["Starbucks"])+(new_df["Nightlife"])+(new_df["transportation"])+(new_df["preschool"])+(new_df["veganRest"])+(new_df["petService"])

In [94]:

new_df["density"] = (new_df["Starbucks"]*1)+(new_df["Nightlife"]*0.8)+(new_df["transportation"]*1)+(new_df["preschool"]*0.7)+(new_df["veganRest"]*0.6)+(new_df["petService"]*0.6)

In [96]:
new_df = new_df.sort_values(by='density', ascending=False)

new_df.sample(4)

,name,city,latitude,longitude,country_code,Starbucks,Nightlife,transportation,preschool,veganRest,petService,Total,density
19,Wonga,London,51.519204,-0.162610,GBR,3,42,14,1,2,1,63,53.1
37,stickK,New York,40.744837,-73.983843,USA,10,50,8,15,17,10,110,84.7
78,ELDR Media,Corte Madera,37.926009,-122.516289,USA,3,1,1,2,2,2,11,8.6
66,Photobucket,Denver,47.601803,-122.332085,USA,1,50,18,5,1,3,78,64.9


# World Heat map
Now we are plotting all the locations we have in our dataset into a heatmap a see the most dense location, in term of companies


In [55]:
companies_group = folium.FeatureGroup(name=f"Companies: {new_df.shape[0]}")

In [56]:
wolrd_map  = Map(location = ["41.382707", "2.169242"], zoom_start = 2)
HeatMap(data=new_df[["latitude", "longitude"]], 
        radius=25,
       gradient = {0.4: 'yellow', 0.65: 'orange', 1: 'white'}).add_to(companies_group)

In [57]:
companies_group.add_to(wolrd_map)

In [58]:
wolrd_map
wolrd_map.save('wolrd_map.html')

# It looks like we're going to New York !
using the ratio/density feature, we are going to sort the data set in an descending order, because we are looking for the place we the most number of amenities.


In [121]:
ny_df = new_df[new_df["city"] == "New York"]



In [122]:
ny_df.to_csv("./dataset/new_york_data.csv", index = False)

In [123]:
nyork_df = pd.read_csv("./dataset/new_york_data.csv")


In [124]:
#nyork_df


,name,city,latitude,longitude,country_code,Starbucks,Nightlife,transportation,preschool,veganRest,petService,Total,density
0,Heavy,New York,40.753385,-73.989620,USA,14,50,26,12,5,8,115,96.2
1,Optimum Energy,New York,40.751892,-73.987592,USA,14,50,19,13,7,10,113,92.3
2,Tripology,New York,40.746883,-73.983969,USA,8,50,13,16,12,10,109,85.4
3,Yipit,New York,40.744618,-73.987764,USA,9,50,12,14,14,10,109,85.2
4,Major League Gaming,New York,40.752672,-73.975930,USA,15,49,10,12,11,10,107,85.2
5,stickK,New York,40.744837,-73.983843,USA,10,50,8,15,17,10,110,84.7
6,Crispy Gamer,New York,40.750597,-73.985273,USA,11,50,13,13,9,10,106,84.5
7,Inform Technologies,New York,40.744639,-73.983963,USA,9,50,8,15,17,10,109,83.7
8,Mimeo,New York,40.745216,-73.982807,USA,8,50,10,14,15,10,107,82.8
9,TradeCard,New York,40.707729,-74.007801,USA,11,50,13,14,7,6,101,81.6


In [112]:
NY_map = Map(location = ["40.751892", "-73.987592"], zoom_start = 10)
folium.Circle([40.751892, -73.987592],radius=1000).add_to(NY_map)


In [113]:
NY_map

In [114]:
#top 10 locations in NY based on density

In [116]:
TOP_NY_map = Map(location = ["40.751892", "-73.987592"], zoom_start = 15)

In [117]:
icon = Icon(color = "blue",
            opacity = 0.1,
            prefix = "fa",
            icon = "briefcase",
            icon_color = "black")

In [118]:
for index, row in top_ny.iterrows():
    
    folium.Circle([row["latitude"],row["longitude"]],radius=1000).add_to(TOP_NY_map)
    folium.Marker(location =[row["latitude"],row["longitude"]],tooltip = row["name"],icon = icon).add_to(TOP_NY_map)
    

In [119]:
TOP_NY_map